1) Mount Drive and define paths

In [1]:
# Mount Drive
from google.colab import drive
drive.mount('/content/drive')

# === Update only if your folder name is different ===
PROJECT_DIR   = "/content/drive/MyDrive/WorstellWitnessWaybackMachine"
NOTEBOOKS_DIR = f"{PROJECT_DIR}/Notebooks"

print("Project dir:", PROJECT_DIR)
print("Notebooks dir:", NOTEBOOKS_DIR)


Mounted at /content/drive
Project dir: /content/drive/MyDrive/WorstellWitnessWaybackMachine
Notebooks dir: /content/drive/MyDrive/WorstellWitnessWaybackMachine/Notebooks


2) Here’s the tiny “watch latest Actions run + ping the site” cell you can drop into any Colab notebook and run.

In [2]:
# Watch the latest "Deploy MkDocs site" run, then ping your Pages URL until it's live

GITHUB_USERNAME = "BillWorstell"
REPO_NAME      = "WorstellWitnessWaybackMachine"
SITE_URL       = "https://billworstell.github.io/WorstellWitnessWaybackMachine/"
GH_TOKEN       = ""   # optional; paste a classic PAT if you hit API rate limits

import time, requests

base = f"https://api.github.com/repos/{GITHUB_USERNAME}/{REPO_NAME}"

def gh_get(path):
    headers = {"Accept": "application/vnd.github+json"}
    if GH_TOKEN:
        headers["Authorization"] = f"token {GH_TOKEN}"
    r = requests.get(f"{base}{path}", headers=headers, timeout=30)
    r.raise_for_status()
    return r.json()

# 1) Find the workflow by path or name
wfs = gh_get("/actions/workflows")
wf = next(
    (w for w in wfs.get("workflows", [])
     if w.get("path") == ".github/workflows/deploy.yml"
        or w.get("name","").lower().startswith("deploy mkdocs")),
    None,
)
if not wf:
    raise SystemExit("Couldn't find workflow '.github/workflows/deploy.yml'")

# 2) Get the latest run on main
runs = gh_get(f"/actions/workflows/{wf['id']}/runs?branch=main&per_page=1").get("workflow_runs", [])
if not runs:
    raise SystemExit("No runs found yet — push a change or click 'Run workflow' in Actions.")

run = runs[0]
sid, url = run["id"], run["html_url"]
print(f"Latest run: {url}")

# 3) Poll until the run completes (up to ~4 minutes)
tries = 48  # 48 * 5s = 4 minutes
while run["status"] != "completed" and tries > 0:
    time.sleep(5)
    run = gh_get(f"/actions/runs/{sid}")
    print(f"… status: {run['status']} / conclusion: {run.get('conclusion')}")

print(f"Final: {run['status']} / {run.get('conclusion')} — {url}")

# 4) If success, ping the Pages site until HTTP 200 (up to ~2 minutes)
if run.get("conclusion") == "success":
    print(f"Pinging {SITE_URL} …")
    live = False
    for i in range(24):        # 24 * 5s = 2 minutes
        try:
            r = requests.get(SITE_URL, timeout=10)
            if r.ok:
                live = True
                print("✅ Site is live!")
                break
            else:
                print(f"   (Attempt {i+1}/24) HTTP {r.status_code}")
        except Exception as e:
            print(f"   (Attempt {i+1}/24) error: {e}")
        time.sleep(5)
    if not live:
        print("⚠️ Site not live yet — give it another minute and refresh.")
else:
    print("❌ Build did not succeed. Open the run URL above and check the failing step.")


Latest run: https://github.com/BillWorstell/WorstellWitnessWaybackMachine/actions/runs/17929884676
Final: completed / success — https://github.com/BillWorstell/WorstellWitnessWaybackMachine/actions/runs/17929884676
Pinging https://billworstell.github.io/WorstellWitnessWaybackMachine/ …
✅ Site is live!
